In [1]:
import socket
import time
import math
import crcmod
from get_crc16 import get_crc16 #own function
from request_cfg2 import request_cfg2 #own function
from decode_cfg2 import decode_cfg2 #own function
from send_command import send_command #own function
from decode_dataframe import decode_dataframe #own function


In [5]:
#PMU idcode
idcode=1
tcpip= "10.10.200.22"
tcpport = 4712
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(5)
s.connect((tcpip, tcpport))

#ask for cfg2 to get the PMU parameters to use with the received dataframe
cfg2 = request_cfg2(s,idcode)

#send command to turn ON data transmission
cmd = 0b0010
send_command(s,idcode,cmd)


#read  measurements 
#this code is not optimal, n frames per second are expected
#is only for testing purposes, data could be lost or superimposed
n= 0
dataframenumber = 0 #counter for dataframes
#ask for n dataframes
while n  < 100:
    #can only recieve 4096 bytes
    #maybe i should use a function that can get me 
    #the numbers of bytes before using the recv function
    data = s.recv(4096)
    #check if first byte is AA (begining of the dataframe)
    if data[:1].hex() =='aa':
        #get the framesize
        framesizebytes  = data[2:4]
        framesize = int.from_bytes(framesizebytes,'big')
        #the dataframes could be superimposed, so i get the framesize and
        #divide the data into n framesize strings
        #split data in chunks of framsize
        if len(data) > framesize:
            datachunks = [data[i:i+n] for i in range(0, len(data), framesize)]
            for d in datachunks:
                #print('Framesize: {}, length of dataframe: {}'.format(framesize,len(d)))
                #print(d)
                #:-2 porque los ultimos bytes son el numero crc
                [_,_,crccalc] =get_crc16(d[:-2])
                crcsrc=int.from_bytes(d[-2:],'big')
                if crccalc==crcsrc:
                    #print('CRC Ok!')
                    pass
                else:
                    print('Error CRC')
                dataframenumber +=1
                
        else: #no superimposed dataframes
            #print('Framesize: {}, length of dataframe: {}'.format(framesize,len(data)))
            #print(data)
            dataframenumber +=1
            #:-2 porque los ultimos bytes son el numero crc
            [_,_,crccalc] =get_crc16(data[:-2])
            crcsrc=int.from_bytes(data[-2:],'big')
            if crccalc==crcsrc:
                #print('CRC Ok!')
                pass
            else:
                print('Error CRC')
        #print('Dataframe number: {}'.format(dataframenumber))
        #print('------------------------------------------------------------')

    
#     print(len(data))
#     print('-----------------------------------------------------------------')
#     print(data.hex().upper())
    
    
    n=n+1
    
    
#send command to turn off data transmission
cmd = 0b0001
send_command(s,idcode,cmd)
s.close()

Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC
Error CRC


In [61]:
#PMU idcode
idcode=1
tcpip= "10.10.200.22"
tcpport = 4712
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(5)
s.connect((tcpip, tcpport))

#ask for cfg2 to get the PMU parameters to use with the received dataframe
cfg2 = request_cfg2(s,idcode)

#send command to turn ON data transmission
cmd = 0b0010
send_command(s,idcode,cmd)


#read  measurements 
#this code is not optimal, n frames per second are expected
#is only for testing purposes, data could be lost or superimposed
n= 0
dataframenumber = 0 #counter for dataframes
#ask for n dataframes
while n  < 5:
    #can only recieve 4096 bytes
    #maybe i should use a function that can get me 
    #the numbers of bytes before using the recv function
    start = time.process_time()
    data = s.recv(4096)
    #check if first byte is AA (begining of the dataframe)
    if data[:1].hex() =='aa':
        #get the framesize
        framesizebytes  = data[2:4]
        framesize = int.from_bytes(framesizebytes,'big')
        #the dataframes could be superimposed, so i get the framesize and
        #divide the data into n framesize strings
        #split data in chunks of framsize
        if len(data) > framesize:
            datachunks = [data[i:i+framesize] for i in range(0, len(data), framesize)]
            for d in datachunks:
                #print('Framesize: {}, length of dataframe: {}'.format(framesize,len(d)))
                #print(d)
                #:-2 porque los ultimos bytes son el numero crc
                [_,_,crccalc] =get_crc16(d[:-2])
                crcsrc=int.from_bytes(d[-2:],'big')
                if crccalc==crcsrc:
                    print('CRC Ok!')
                    print(d)
                    pass
                else:
                    print('Error CRC')
                    print('data len: {} framesize:{}'.format(len(data),framesize))
                    print(data.hex())
                    cmd = 0b0001
                    send_command(s,idcode,cmd)
                    s.close()
                    endt = time.process_time()
                    tp = endt-start
                    print('Error tp : {}'.format(tp))
                    break
                dataframenumber +=1
                
        else: #no superimposed dataframes
            #print('Framesize: {}, length of dataframe: {}'.format(framesize,len(data)))
            #print(data)
            dataframenumber +=1
            #:-2 porque los ultimos bytes son el numero crc
            [_,_,crccalc] =get_crc16(data[:-2])
            crcsrc=int.from_bytes(data[-2:],'big')
            if crccalc==crcsrc:
                #print('CRC Ok!')
                pass
            else:
                print('Error CRC')
                print(data)
        #print('Dataframe number: {}'.format(dataframenumber))
        #print('------------------------------------------------------------')  
#     print(len(data))
#     print('-----------------------------------------------------------------')
#     print(data.hex().upper())
    n=n+1
    endt = time.process_time()
    tp = endt-start
    if tp !=0:
        print('Final tp : {}'.format(tp))
#send command to turn off data transmission
cmd = 0b0001
send_command(s,idcode,cmd)
s.close()

In [52]:
datachunks = [data[i:i+framesize] for i in range(0, len(data), framesize)]
datachunks

[b"\xaa\x01\x00|\x00\x01\x00\x00'?\x0f\x0fB@!\xb0=P\xdey@>\x8eK>\x1b\xd6\xc0@?5\xda;\x93\xe0\x00\xbe\x15\xe0/:\xde0\x00\xbe\x0c%\xfb;\xae\xc8\x00\xbe\x11\xa069\x13\xc0\x00@6T\x8c;5\xa0\x00\xbe\x0c\xa06:\xb2p\x00@=9\xe4BrEHB\x95~*\xb4=\xee\xd14\xad\x8f\x804\x91F<\xbf\x0c\x13\x13\xba1\xc3#:1\xc4\x886\xb2\x0c\x19\xbf\x7f\xfd\xfe\x00\xff\xef\x92",
 b'\xaa\x01\x00|\x00\x01\x00\x00\'?\x0f\x1e\x84\x80!\xb0=P\xfd-@@\xf3\xf8>\x1b\xee_@A\x9b\x89;\x940\x00\xbd\xdf\n::\xde\x00\x00\xbd\xcb\ni;\xae\xc0\x00\xbd\xd6\xca|9\x17\x00\x00@8\xb6\x8f;6\x00\x00\xbd\xcd\n|:\xb2p\x00@?\x93\xf0Br\x83|\xc2_\xca\xc8\xb3!\xad\x164\x8c\x81\xf74\x8b\x0b\xf4\xbe\x13H\xb9\xba2A":2B\x886\xb2\xb2`\xbf\x7f\xfd\xfe\x00\xff\x85\xc6']

In [63]:
def read_data(sockobj,buffersize=2048):
    #read  measurements 
    #this code is not optimal, n frames per second are expected
    #is only for testing purposes, data could be lost or superimposed
    #ask for n dataframes

    #can only recieve buffersize bytes
    #maybe i should use a function that can get me 
    #the numbers of bytes before using the recv function
    data = s.recv(buffersize)
    #check if first byte is AA (begining of the dataframe)
    if data[:1].hex() =='aa':
        #get the framesize
        framesizebytes  = data[2:4]
        framesize = int.from_bytes(framesizebytes,'big')
        #the dataframes could be superimposed, so i get the framesize and
        #divide the data into n framesize strings
        #split data in chunks of framsize
        if len(data) > framesize:
            datachunks = [data[i:i+framesize] for i in range(0, len(data), framesize)]
            for d in datachunks:
                #print('Framesize: {}, length of dataframe: {}'.format(framesize,len(d)))
                #print(d)
                #:-2 porque los ultimos bytes son el numero crc
                [_,_,crccalc] =get_crc16(d[:-2])
                crcsrc=int.from_bytes(d[-2:],'big')
                if crccalc==crcsrc:
                    #print('CRC Ok!')
                    #print('Dataframe read:')
                    #print(d)
                    return d, True
                    
                else:
#                     print('Error CRC')
#                     print('data len: {} framesize:{}'.format(len(data),framesize))
#                     print(data.hex())
                    return data, False

        else: #no superimposed dataframes
            #print('Framesize: {}, length of dataframe: {}'.format(framesize,len(data)))
            #print(data)
            #:-2 porque los ultimos bytes son el numero crc
            [_,_,crccalc] =get_crc16(data[:-2])
            crcsrc=int.from_bytes(data[-2:],'big')
            if crccalc==crcsrc:
                #print('CRC Ok!')
                return data, True
            else:
                #print('Error CRC')
                #print(data)
                return data, False
        #print('Dataframe number: {}'.format(dataframenumber))
        #print('------------------------------------------------------------')  
#     print(len(data))
#     print('-----------------------------------------------------------------')
#     print(data.hex().upper())



In [70]:
#PMU idcode
idcode=1
tcpip= "10.10.200.22"
tcpport = 4712
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(5)
s.connect((tcpip, tcpport))

#ask for cfg2 to get the PMU parameters to use with the received dataframe
cfg2 = request_cfg2(s,idcode)

#send command to turn ON data transmission
cmd = 0b0010
send_command(s,idcode,cmd)


#read  measurements 
#this code is not optimal, n frames per second are expected
#is only for testing purposes, data could be lost or superimposed
n= 0
dataframenumber = 0 #counter for dataframes
#ask for n dataframes
for i in range(5):
    [dframe,crcok] = read_data(s)
    print('Dataframe # {}:-----------------------------------------'.format(i+1))
    print(dframe.hex())
    print('CRC OK: {}'.format(crcok))
#send command to turn off data transmission
cmd = 0b0001
send_command(s,idcode,cmd)
s.close()

Dataframe # 1:-----------------------------------------
aa01007c0001000000440f0f424020d03d52ad5040397fb23e1d3140403a26bf3b94f000be6731cb3adf0000be5bf1d73bafb800be6331d53919000140319fc53b371000be5df1d23ab4800040381d1d4272435f414053a533d33bfa33d98623b2cfb8a83f789908ba349d433a349e8e36acd05cbf7ffe2b00ff6afb
CRC OK: True
Dataframe # 2:-----------------------------------------
aa01007c0001000000440f1e848020d03d52845b40384c2d3e1d11bf4038f31a3b94e000be7a0c2b3adf0fffbe6f4c213bafd800be762c1c3914c0004030ae203b369800be70cc173ab540004036eb79427284f8c2602b18b41113a2341b079f335a9d00bf6f9085ba34636e3a3464d936b4f259bf7ffdfd00ff9c97
CRC OK: True
Dataframe # 3:-----------------------------------------
aa01007c0001000000440f2dc6c020d03d52c2fd403b03263e1d4100403baa183b951000be4e3c2a3adf5000be443c423bafa400be4a5c353915c0004032dce13b36b801be453c323ab4400040399c8042728503410d0f7bb51e97d2351f149d33471ae2bf7f372dba34d5673a34d6f636bddf6ebf7ffdcb00ffd66f
CRC OK: True
Dataframe # 4:--------------------------------